In [1]:
#pip install plotly_express==0.4.0

In [54]:
from google.cloud import bigquery
import pandas_gbq
import pandas as pd
import numpy as np
import requests
import datetime
import re
import json
from datetime import date, timedelta
from datetime import datetime
import pyspark as ps
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr,first, last,when, split, col,lit, concat, date_format,to_utc_timestamp,to_timestamp, regexp_replace
client = bigquery.Client(location="us-central1")
print("Client creating using default project: {}".format(client.project))
pd.set_option('display.width', 1000)
pd.set_option("max_colwidth",10000)
pd.set_option("max_rows",1000)
pd.set_option("max_columns",100)

Client creating using default project: saas-analytics-io


In [55]:
spark = SparkSession.builder.appName("gomoze").getOrCreate()

In [62]:
df_metadata = spark.read.format('bigquery').option('project','saas-analytics-io').option('table','yt1_v1data1.metadata').option("filter", """tenantCode = 'CHIP' and date(time) between '2023-04-10' and 
   '2023-04-15' """).load()

In [63]:
df_udf = spark.read.format('bigquery').option('project','saas-analytics-io').option('table','yt1_v1data1.udf').option("filter", """tenantCode = 'CHIP' and date(time) between '2023-04-10' and 
   '2023-04-15' """).load()

In [64]:
df_metadata = df_metadata.withColumn('id_serial_number',concat(df_metadata['id'],lit('-'), df_metadata['serialNumber']))
df_udf = df_udf.withColumn('id_serial_number',concat(df_udf['metadataId'],lit('-'), df_udf['serialNumber']))
df_udf = df_udf.distinct()

In [65]:
df_pivot = df_udf.groupBy("id_serial_number").pivot("key").agg(first("value")).fillna('N/A')

In [66]:
df_full = df_metadata.join(df_pivot,['id_serial_number'],"outer")
df_full = df_full.fillna('N/A')

In [67]:
df_pivot.printSchema()

root
 |-- id_serial_number: string (nullable = false)
 |-- CountedDate: string (nullable = false)
 |-- PartialCount: string (nullable = false)
 |-- PartialCountAsPercent: string (nullable = false)
 |-- PartialCountAsQty: string (nullable = false)
 |-- PartialCountDate: string (nullable = false)
 |-- PartialCountFirstDate: string (nullable = false)
 |-- PartialCountUOM: string (nullable = false)
 |-- Retail_Action: string (nullable = false)
 |-- Retail_AlertType: string (nullable = false)
 |-- Retail_BizTransactionId: string (nullable = false)
 |-- Retail_Bizlocation: string (nullable = false)
 |-- Retail_Bizlocation_Original: string (nullable = false)
 |-- Retail_Bizstep: string (nullable = false)
 |-- Retail_CC_RetrieveFlag: string (nullable = false)
 |-- Retail_CC_SyncEndFlag: string (nullable = false)
 |-- Retail_Color: string (nullable = false)
 |-- Retail_Commissioned: string (nullable = false)
 |-- Retail_CycleCountDate: string (nullable = false)
 |-- Retail_DisplayName: string (

In [68]:
df_pivot.show()

+--------------------+-----------+------------+---------------------+-----------------+----------------+---------------------+---------------+-------------+----------------+-----------------------+------------------+---------------------------+--------------+----------------------+---------------------+------------+-------------------+---------------------+------------------+------------------+-----------------------+--------------------+----------------+--------------------+-------------+--------------------+-----------+------------------+------------------+---------------+--------------+--------------------+--------------------+------------------------+-----------------------+---------------------+-------------------------+-------------------------+-------------------------+-------------------------+-------------------+------------------+--------------------+------------------+------------------+--------------------------------+---------------------------------+----------------------

In [119]:
df_full_s = df_full[['id_serial_number','Retail_Object_Type','time','Retail_RecordTime','Retail_Bizstep','Retail_Disposition','Retail_ReadPoint','Retail_BizLocation',
                     'Retail_Premise','Retail_BizTransactionId','Retail_Extension','Retail_Action','Retail_EPC_PureIdentity','Retail_Product_UPC','sku','serialNumber',
                     'thingType','bridgeKey','specName','Retail_Product_Weight_Gross_Unit','Retail_Product_Weight_Gross_Value',
                     'logicalReader','lotNumber','packagingDate','source','suppliername','zone']]


In [120]:
df_full_s = df_full_s.withColumnRenamed('id_serial_number','id')
df_full_s = df_full_s.withColumnRenamed('Retail_Object_Type','Object_Type')
df_full_s = df_full_s.withColumnRenamed('time','eventTime')
df_full_s = df_full_s.withColumnRenamed('Retail_Bizstep','bizStep')
df_full_s = df_full_s.withColumnRenamed('Retail_Disposition','disposition')
df_full_s = df_full_s.withColumnRenamed('Retail_readPoint','readPoint')
df_full_s = df_full_s.withColumnRenamed('Retail_BizLocation','bizLocation')
df_full_s = df_full_s.withColumnRenamed('Retail_Premise','premise')
df_full_s = df_full_s.withColumnRenamed('Retail_BizTransactionId','bizTransList_value')
df_full_s = df_full_s.withColumnRenamed('Retail_Extension','extension')
df_full_s = df_full_s.withColumnRenamed('Retail_Action','action')
df_full_s = df_full_s.withColumnRenamed('Retail_EPC_PureIdentity','epcList_epc')
df_full_s = df_full_s.withColumnRenamed('Retail_Product_UPC','epcList_gtin')
df_full_s = df_full_s.withColumnRenamed('sku','epcList_sku')
df_full_s = df_full_s.withColumnRenamed('serialNumber','epcList_hexa')

In [121]:
df_full_s.printSchema()

root
 |-- id: string (nullable = false)
 |-- Object_Type: string (nullable = false)
 |-- eventTime: timestamp (nullable = true)
 |-- Retail_RecordTime: string (nullable = false)
 |-- bizStep: string (nullable = false)
 |-- disposition: string (nullable = false)
 |-- readPoint: string (nullable = false)
 |-- bizLocation: string (nullable = false)
 |-- premise: string (nullable = false)
 |-- bizTransList_value: string (nullable = false)
 |-- extension: string (nullable = false)
 |-- action: string (nullable = false)
 |-- epcList_epc: string (nullable = false)
 |-- epcList_gtin: string (nullable = false)
 |-- epcList_sku: string (nullable = false)
 |-- epcList_hexa: string (nullable = false)
 |-- thingType: string (nullable = false)
 |-- bridgeKey: string (nullable = false)
 |-- specName: string (nullable = false)
 |-- Retail_Product_Weight_Gross_Unit: string (nullable = false)
 |-- Retail_Product_Weight_Gross_Value: string (nullable = false)
 |-- logicalReader: string (nullable = false)


In [122]:
#df_full.printSchema()

In [123]:
#df_full_pd = df_full.toPandas()

In [124]:
#df_full_pd.count()['id_serial_number']

In [125]:
#df_full_pd.head()

In [126]:
#df_full_pd.groupby(['thingType','Retail_Bizstep']).count()

In [127]:
#df_full_s = df_full_pd[['id_serial_number','time','thingType','bridgeKey','specName','Retail_Bizstep','Retail_Disposition'
 #                      ,'Retail_Object_Type','Retail_Premise','Retail_Product_UPC','Retail_Product_Weight_Gross_Unit','Retail_Product_Weight_Gross_Value'
  #                     ,'Retail_ReadPoint','logicalReader','lotNumber','packagingDate','source','suppliername','zone']]

In [128]:
#df_full_s.head()

### Location ENDPOINT

In [129]:
headers = {
  'Content-Type': 'application/json',
  'tenant': 'CHIPscg',
  'apiKey':'BHUMJQUXGYFF5AMR'
        }
data = {"filters":[{
                 "values":[
                 ]}
           ]}


response = requests.get('https://chipotle.mojixretail.io/statemachine-api-configuration/rest/configuration/locations?level=premise', headers=headers, data=json.dumps(data))
df_premise = pd.json_normalize(json.loads(response.text))
df_premise_sp = spark.createDataFrame(df_premise.astype(str))


In [130]:
df_premise.count()

id                          340
code                        340
name                        340
level                       340
type                        340
latitude                    340
longitude                   340
sgln                          0
bizLocation                 340
bizLocationRegularizer      340
hotspotZOffset              340
attributes                  340
addressLine1                340
addressLine2                  0
addressLine3                  0
addressPostalCode           340
addressCity                 340
addressState                340
addressCountry              340
parent                      340
hierarchy.organizationId    340
hierarchy.premiseId           0
hierarchy.floorId             0
hierarchy.areaId              0
hierarchy.zoneId              0
dtype: int64

In [131]:
#df_premise.head()

In [132]:
df_premise_s = df_premise_sp[['code','name','level','type','latitude','longitude','addressCity','addressState','addressCountry']]

### Product ENDPOINT

In [133]:
headers = {
  'Accept': 'application/json, text/plain, */*',   
  'Content-Type': 'application/json',
  'tenant': 'CHIPscg',
  'apiKey':'BHUMJQUXGYFF5AMR'
        }
data = {"filters":[{
                 "values":[
                 ]}
           ]}


response = requests.get('https://chipotle.mojixretail.io/product-api/rest/products/', headers=headers,)
df_product = pd.json_normalize(json.loads(response.text))
df_product_sp = spark.createDataFrame(df_product.astype(str))

In [146]:
df_product.count()

gtin                   1113
updateTime             1113
creationTime           1113
productCode            1113
productLabelLong       1113
categoryParent         1113
sizeLabel                 1
brandCode                 1
brandLabel             1108
modelLabel             1062
colorLabel             1058
prices                 1113
displayGtin            1112
colorCode              1058
imageURL                 80
contentUOM               88
contentQty               89
packagingUOM             87
categoryLevel1Label      87
categoryLevel1Code       87
categoryLevel2Label      82
categoryLevel3Label      82
categoryLevel4Label      82
contentGTIN              32
parentGTIN               31
dtype: int64

In [135]:
df_product_s = df_product_sp[['gtin','productLabelLong','categoryParent','brandLabel','modelLabel','colorCode','imageURL','contentQty'
                          ,'packagingUOM','categoryLevel1Label','categoryLevel2Label','categoryLevel3Label','categoryLevel4Label']]

In [136]:
#df_product[df_product['gtin']=='10732966109943']

In [137]:
#df_full_s = pd.merge(df_full_s,df_premise_s, how ='left' , left_on='Retail_Premise', right_on = 'code')
df_full_s = df_full_s.join(df_premise_s,df_full_s.premise == df_premise_s.code,"Left")

In [138]:
#df_full_s = pd.merge(df_full_s,df_product_s, how ='left' , left_on='Retail_Product_UPC', right_on = 'gtin')
df_full_s = df_full_s.join(df_product_s,col('epcList_gtin') == col('gtin'),"Left")

In [139]:
df_full_s.printSchema()

root
 |-- id: string (nullable = false)
 |-- Object_Type: string (nullable = false)
 |-- eventTime: timestamp (nullable = true)
 |-- Retail_RecordTime: string (nullable = false)
 |-- bizStep: string (nullable = false)
 |-- disposition: string (nullable = false)
 |-- readPoint: string (nullable = false)
 |-- bizLocation: string (nullable = false)
 |-- premise: string (nullable = false)
 |-- bizTransList_value: string (nullable = false)
 |-- extension: string (nullable = false)
 |-- action: string (nullable = false)
 |-- epcList_epc: string (nullable = false)
 |-- epcList_gtin: string (nullable = false)
 |-- epcList_sku: string (nullable = false)
 |-- epcList_hexa: string (nullable = false)
 |-- thingType: string (nullable = false)
 |-- bridgeKey: string (nullable = false)
 |-- specName: string (nullable = false)
 |-- Retail_Product_Weight_Gross_Unit: string (nullable = false)
 |-- Retail_Product_Weight_Gross_Value: string (nullable = false)
 |-- logicalReader: string (nullable = false)


In [140]:
#df_full_s[df_full_s['thingType']=='ITEM'].fillna('N/A').groupby(['Retail_Bizstep','level','addressState','modelLabel','suppliername','logicalReader']).count()[['id_serial_number']]

In [145]:
df_full_s.groupBy('extension').count().show()

+--------------------+-----+
|           extension|count|
+--------------------+-----+
|{"mjx_user":"cala...| 2994|
|{"mjx_user":"fiel...|  256|
|{"mjx_user":"lanc...|  108|
|{"mjx_user":"osi"...|   16|
|{"mjx_user":"aver...|   80|
|{"mjx_user":"osi"...|   16|
|{"mjx_user":"pilg...|   66|
|{"mjx_user":"aver...|    2|
|{"mjx_user":"aver...|    2|
|{"mjx_user":"fiel...|  240|
|{"mjx_user":"miss...|    2|
|{"mjx_deviceId":"...|    2|
|{"mjx_user":"aver...|    2|
|{"mjx_user":"osi"...|    4|
|{"mjx_user":"aver...|  120|
|{"mjx_user":"aver...|   30|
|{"mjx_deviceId":"...|    2|
|{"mjx_deviceId":"...|    2|
|{"mjx_user":"cala...|  100|
|{"mjx_deviceId":"...|    2|
+--------------------+-----+
only showing top 20 rows



In [141]:
#table = "saas-analytics-io.GoMoZe.chipotle_events_locations_products"
#df_full_s.to_gbq(table, if_exists='replace')



bucket = "finops-outputs"
spark.conf.set('temporaryGcsBucket', bucket)

df_full_s.write.format('bigquery') \
    .option('table', 'saas-analytics-io.GoMoZe.CHIP_events_enriched') \
    .mode('overwrite') \
    .save()


df_premise_s.write.format('bigquery') \
    .option('table', 'saas-analytics-io.GoMoZe.CHIP_locations') \
    .mode('overwrite') \
    .save()

In [77]:
df_premise.head()

,id,code,name,level,type,latitude,longitude,sgln,bizLocation,bizLocationRegularizer,hotspotZOffset,attributes,addressLine1,addressLine2,addressLine3,addressPostalCode,addressCity,addressState,addressCountry,parent,hierarchy.organizationId,hierarchy.premiseId,hierarchy.floorId,hierarchy.areaId,hierarchy.zoneId
0,1001,0840662199991,Mojix – 9999,premise,store,1.000000,1.000000,None,urn:epc:id:sgln:08406621.9999.0,urn:epc:id:sgln:08406621.9999.0,0,"inventory_app_flow_transferout_receiving,csv_item_process_disposition,csv_item_process_bizStep,asn_auto_on","La Paz, Bolivia",None,None,00000,La Paz,LP,BO,1000,1000,None,None,None,None
1,1006,0081953000002,"Ed Miniat Inc.-South Holland, IL",premise,supplier,41.598330,-87.631004,None,urn:epc:id:sgln:0081953.00000.0,urn:epc:id:sgln:0081953.00000.0,0,"inventory_app_flow_transferout_receiving,asn_auto_on",16250 S. Vincennes Ave,None,None,60473,South Holland,IL,US,1000,1000,None,None,None,None
2,1011,0840662100423,Clybourn Commons - 42,premise,store,41.917336,-87.661150,None,urn:epc:id:sgln:08406621.0042.0,urn:epc:id:sgln:08406621.0042.0,0,"inventory_app_flow_transferout_receiving,asn_auto_on",2000 N. Clybourn Ave.,None,None,60614,Chicago,IL,US,1000,1000,None,None,None,None
3,1016,0840662100539,Streets of Woodfield - 53,premise,store,42.040253,-88.035934,None,urn:epc:id:sgln:08406621.0053.0,urn:epc:id:sgln:08406621.0053.0,0,"inventory_app_flow_transferout_receiving,asn_auto_on","601 N. Martingale Rd., Suite 330",None,None,60173,Schaumburg,IL,US,1000,1000,None,None,None,None
4,1021,0840662100911,Franklin - 91,premise,store,41.885563,-87.637405,None,urn:epc:id:sgln:08406621.0091.0,urn:epc:id:sgln:08406621.0091.0,0,"inventory_app_flow_transferout_receiving,asn_auto_on",233-235 W. Lake St.,None,None,60606,Chicago,IL,US,1000,1000,None,None,None,None


In [ ]:
#df_product[df_product['gtin']=='761010282830']
#df_product[df_product['gtin']=='761010282564']

In [88]:
#response = requests.get('https://api.openweathermap.org/data/2.5/weather?q=South Holland&appid=29663d4acb6fa3918ab5ae699c2c0d72&units=metric')
#df_weather_2 = pd.json_normalize(json.loads(response.text))

response = requests.get('https://archive-api.open-meteo.com/v1/era5?latitude=41.598330&longitude=-87.631004&start_date=2023-04-10&end_date=2023-04-13&hourly=temperature_2m')
df_weather_3 = pd.json_normalize(json.loads(response.text))


In [25]:
df_weather_s = pd.DataFrame()
#df_weather_full = pd.DataFrame()
#df_premise_weather = pd.DataFrame()
#df_premise_weather_full = pd.DataFrame()

for i in range (276 , df_premise.count()['code']): 
    lat = df_premise['latitude'][i]
    long = df_premise['longitude'][i]
    code = df_premise['code'][i]
    url = 'https://archive-api.open-meteo.com/v1/era5?latitude=%s&longitude=%s&start_date=2023-04-10&end_date=2023-04-13&hourly=temperature_2m'%(lat,long)
    response = requests.get(url)
    print(i)
    for j in range (0 , len(response.json()['hourly']['temperature_2m'])): 
        df_weather_s.loc[j,'hour'] = response.json()['hourly']['time'][j]
        df_weather_s.loc[j,'temperature'] = response.json()['hourly']['temperature_2m'][j]
        df_weather_s.loc[:,'code'] = code
        
    df_weather_full = df_weather_full.append(df_weather_s) 
    
    #df_premise_weather = pd.merge(df_premise[['id','code','name','latitude','longitude','addressCountry','addressCity','addressLine1']],df_weather_s,
     #                            how= 'inner', on= 'code')
    #df_premise_weather_full = df_premise_weather_full.append(df_premise_weather)
    

276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339


In [30]:
df_chip_enrich_data = spark.read.format('bigquery').option('project','saas-analytics-io').option('table','GoMoZe.CHIP_events_enriched').load()

In [40]:
df_chip_enrich_data = df_chip_enrich_data.withColumn('datehour', date_format(df_chip_enrich_data['time'], "d/M/y H"))
df_chip_enrich_data = df_chip_enrich_data.withColumn('datehour', to_utc_timestamp(to_timestamp(df_chip_enrich_data['datehour'],'d/M/y H'), 'UTC'))

In [31]:
df_weather_full_sp = spark.createDataFrame(df_weather_full)
df_weather_full_sp = df_weather_full_sp.withColumn("datehour",to_timestamp("hour"))

In [43]:
df_chip_enrich_data_weather = df_chip_enrich_data.join(df_weather_full_sp,['datehour','code'],"Left")

In [45]:

bucket = "finops-outputs"
spark.conf.set('temporaryGcsBucket', bucket)

df_chip_enrich_data_weather.write.format('bigquery') \
    .option('table', 'saas-analytics-io.GoMoZe.CHIP_events_enriched_weather') \
    .mode('append') \
    .save()

In [147]:
df_total = spark.read.format('bigquery').option('project','saas-analytics-io').option('table','processed.vt1_total_cost').option("filter", """date_LA_2 between '2023-04-01 00:00:00' and '2023-05-01 23:59:59' """).load()

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 1207, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 1033, in send_command
    response = connection.send_command(command)
  File "/usr/lib/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 1211, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while receiving
ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:40137)
Traceback (most recent call last):
  File "/opt/conda/miniconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3505, in run_c

Py4JError: An error occurred while calling o62.read